In [4]:
from scipy.io import arff
from scipy import stats
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_fscore_support

from IPython.display import HTML

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(palette = "Set2")

In [5]:
def update_plot_params():
    params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
    plt.rcParams.update(params)
    
update_plot_params()

In [23]:
data, _ = arff.loadarff('../data/raw/landsat.arff.txt')
landsat = pd.DataFrame(data)

In [ ]:
sns.pairplot(landsat, hue="class")
plt.savefig('../plots/pairplot.png', dpi=200)
plt.close()

KeyboardInterrupt: 

In [ ]:
sns.heatmap(landsat.iloc[:3,].corr(), annot=True, fmt=".2f")
plt.savefig('../plots/correlation.png', dpi=200)
plt.close()

In [ ]:
# pixel 5

In [24]:
Y = landsat.pop('class')
X = landsat

### Train model 

In [8]:
naive_bayes = GaussianNB()

In [25]:
X = X.astype(int)

In [34]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(np.array(X))

lb.classes_

#lb.transform([1, 6])

ValueError: Multioutput target data is not supported with label binarization

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer  
y=MultiLabelBinarizer().fit_transform(X)  


In [44]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X)  


#enc.n_values_

#enc.feature_indices_
x = enc.transform(X).toarray()

In [50]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
scores = cross_validate(naive_bayes, x, Y,
            cv=5)    

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

In [ ]:
stats = []
for i in [5, 10, 100, 1000]:
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(x_train, y_train)
    knn_pred = knn_model.predict(x_test)
    knn_score = knn_model.score(x_test, y_test) # score = correct predictions / total number of data points
    stats.append([i, knn_score])
    print('Neighbours amount: {}   Score: {}'.format(i, knn_score))
    knn_prfs_score = metrics.precision_recall_fscore_support(y_test, knn_pred)
    knn_df = pd.DataFrame(data={'Precision': knn_prfs_score[0], 'Recall': knn_prfs_score[1], 
                                'F1 score': knn_prfs_score[2], 'Support': knn_prfs_score[3]},
                          columns=['Precision', 'Recall', 'F1 score', 'Support'], index=[0, 1, 2])

    results = (knn_df.style.set_table_styles(styles))
    display(results)